In [118]:
%pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.


In [119]:
from elasticsearch import Elasticsearch
import fitz
import os

In [120]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=["elastic","uolb_G+9wDf4UO+vF-y*"],
    ca_certs="D:\\elastic_search\\config\\certs\\http_ca.crt"
)

es.ping()

True

In [121]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return "Error getting the file"

In [122]:
# Function to check if a file has already been ingested
def is_file_ingested(file_name, es_client, es_index):
    query = {
        "query": {
            "term": {
                "file_name.keyword": file_name
            }
        }
    }
    res = es_client.search(index=es_index, body=query)
    return len(res['hits']['hits']) > 0

In [123]:
es_index = "harry_potter"

In [124]:
!curl -X PUT "localhost:9200/harry_potter" -H 'Content-Type: application/json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (52) Empty reply from server
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: application


In [125]:
print(pdf_folder)

.\pdf\harry_potter


In [126]:
# Function to prepare data for Elasticsearch
def prepare_data_for_elasticsearch(pdf_folder):
    data_list = []
    for root, _, files in os.walk(pdf_folder):
        for file in files:
            if file.lower().endswith('.pdf'):
                if not is_file_ingested(file, es, es_index):
                    pdf_path = os.path.join(root, file)
                    print(file)
                    text_content = extract_text_from_pdf(pdf_path)
                    data = {
                        "file_name": file,
                        "location": pdf_path,
                        "content": text_content
                    }
                    data_list.append(data)
                else:
                    print("file already ingested.Skipping")
    return data_list


In [127]:
print(es_index)

harry_potter


In [128]:
# Function to index data into Elasticsearch
def index_data_to_elasticsearch(data_list, es_index, es_client):
    for data in data_list:
        es_client.index(index=es_index, body=data)

In [129]:
es.ping()

True

In [130]:
pdf_folder = ".\\pdf\\harry_potter" 
es_host = "http://localhost:9200"  
es_index = "harry_potter" 

# Initialize Elasticsearch client
# es_client = Elasticsearch([es_host])
# es is es client

# Extract data from PDFs and prepare for Elasticsearch
data_list = prepare_data_for_elasticsearch(pdf_folder)

# Index data into Elasticsearch
index_data_to_elasticsearch(data_list, es_index, es)

print(f"Indexed {len(data_list)} documents into Elasticsearch index '{es_index}'.")


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [harry_potter]', harry_potter, index_or_alias)

In [ ]:
# ### creta new index in elastic search
# from indexMapping import indexMapping
# try:
#     es.indices.create(index="all_products", mappings=indexMapping)
# except Exception as e:
#     print(e)

In [ ]:
input_search_term = "Safety"
# print("searching...")
# print(vector_of_input_keyword)

body = {
        "query": {
            "multi_match": {
                "query": input_search_term,
                "fields": ["content", "file_name"]
            }
        },
       }
res = es.search(index=es_index, body=body)

results = res["hits"]["hits"]
print(results)


[{'_index': 'pdf_documents', '_id': 'ZuiHuI8Bs1_zjc2DYj5K', '_score': 1.078742, '_source': {'file_name': 'Institution of Safety Alert Signal.pdf', 'location': '.\\pdf\\Safety Policy PDF Scan Files\\Safety Policy Letters\\Institution of Safety Alert Signal.pdf', 'content': ' \n \n \n \n \n \n'}}, {'_index': 'pdf_documents', '_id': 'auiHuI8Bs1_zjc2DYj5q', '_score': 1.078742, '_source': {'file_name': 'Safety related orders policies SOP.pdf', 'location': '.\\pdf\\Safety Policy PDF Scan Files\\Safety Policy Letters\\Safety related orders policies SOP.pdf', 'content': ' \n \n \n \n'}}, {'_index': 'pdf_documents', '_id': 'uuiHuI8Bs1_zjc2DSz2j', '_score': 1.0707239, '_source': {'file_name': 'Conduct of Safety Audits HQSNC letter Safety 8040 6 dated 08 May 15.pdf', 'location': '.\\pdf\\Safety Policy PDF Scan Files\\Conduct of Safety Audits HQSNC letter Safety 8040 6 dated 08 May 15.pdf', 'content': ''}}, {'_index': 'pdf_documents', '_id': '5OiHuI8Bs1_zjc2DUT1_', '_score': 0.9848249, '_source': 